# 员工薪资预测数据分析

## 数据概述
本数据集包含员工的工作经验和薪资信息，用于构建薪资预测模型。

### 数据字段
- **YearsExperience**: 工作经验年限
- **Salary**: 薪资

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats

# 设置显示中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 读取数据
df = pd.read_csv('Employee_Salary_Prediction_Dataset.csv')

# 显示前几行数据
print("数据集形状:", df.shape)
print("\n前5行数据:")
df.head()

In [ ]:
# 数据基本信息
print("数据类型:")
print(df.dtypes)
print("\n缺失值统计:")
print(df.isnull().sum())
print("\n数据描述统计:")
print(df.describe())

## 数据分析结果

### 1. 薪资分布分析

In [ ]:
# 薪资统计
salary_stats = {
    '平均薪资': df['Salary'].mean(),
    '中位数薪资': df['Salary'].median(),
    '最低薪资': df['Salary'].min(),
    '最高薪资': df['Salary'].max(),
    '标准差': df['Salary'].std()
}

print("薪资统计:")
for key, value in salary_stats.items():
    print(f"{key}: ${value:,.2f}")

In [ ]:
# 薪资分布可视化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 直方图
axes[0, 0].hist(df['Salary'], bins=20, edgecolor='black', color='steelblue')
axes[0, 0].set_title('薪资分布直方图')
axes[0, 0].set_xlabel('薪资')
axes[0, 0].set_ylabel('频数')
axes[0, 0].axvline(df['Salary'].mean(), color='red', linestyle='--', label=f'平均值: ${df["Salary"].mean():,.0f}')
axes[0, 0].legend()

# 箱线图
axes[0, 1].boxplot(df['Salary'], vert=True)
axes[0, 1].set_title('薪资箱线图')
axes[0, 1].set_ylabel('薪资')

# Q-Q图
stats.probplot(df['Salary'], dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('薪资Q-Q图')

# 核密度估计
sns.kdeplot(df['Salary'], fill=True, ax=axes[1, 1])
axes[1, 1].set_title('薪资核密度估计')
axes[1, 1].set_xlabel('薪资')
axes[1, 1].set_ylabel('密度')

plt.tight_layout()
plt.show()

### 2. 工作经验分析

In [ ]:
# 工作经验统计
experience_stats = {
    '平均经验': df['YearsExperience'].mean(),
    '中位数经验': df['YearsExperience'].median(),
    '最小经验': df['YearsExperience'].min(),
    '最大经验': df['YearsExperience'].max(),
    '标准差': df['YearsExperience'].std()
}

print("工作经验统计:")
for key, value in experience_stats.items():
    print(f"{key}: {value:.2f}年")

In [ ]:
# 工作经验分布
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['YearsExperience'], bins=15, edgecolor='black', color='coral')
plt.title('工作经验分布')
plt.xlabel('工作经验（年）')
plt.ylabel('频数')

plt.subplot(1, 2, 2)
sns.boxplot(y=df['YearsExperience'], color='lightgreen')
plt.title('工作经验箱线图')
plt.ylabel('工作经验（年）')

plt.tight_layout()
plt.show()

### 3. 经验与薪资关系分析

In [ ]:
# 相关性分析
correlation = df['YearsExperience'].corr(df['Salary'])
print(f"工作经验与薪资的相关系数: {correlation:.4f}")

# 按经验分组统计
df['Experience_Bin'] = pd.cut(df['YearsExperience'], bins=[0, 2, 5, 8, 11], 
                               labels=['0-2年', '2-5年', '5-8年', '8年以上'])

salary_by_experience = df.groupby('Experience_Bin')['Salary'].agg(['mean', 'median', 'count'])
print("\n按经验分组的薪资统计:")
print(salary_by_experience)

In [ ]:
# 散点图
plt.figure(figsize=(12, 6))
plt.scatter(df['YearsExperience'], df['Salary'], alpha=0.6, color='steelblue', s=100)
plt.title('工作经验与薪资关系')
plt.xlabel('工作经验（年）')
plt.ylabel('薪资')
plt.grid(True, alpha=0.3)

# 添加趋势线
z = np.polyfit(df['YearsExperience'], df['Salary'], 1)
p = np.poly1d(z)
plt.plot(df['YearsExperience'], p(df['YearsExperience']), "r--", 
         label=f'趋势线 (斜率: {z[0]:.0f})')
plt.legend()
plt.show()

In [ ]:
# 按经验分组的薪资对比
plt.figure(figsize=(10, 6))
salary_by_experience['mean'].plot(kind='bar', yerr=salary_by_experience['median'] - salary_by_experience['mean'], 
                                   capsize=5, color='steelblue', alpha=0.7)
plt.title('不同经验阶段的平均薪资')
plt.xlabel('工作经验')
plt.ylabel('平均薪资')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 4. 薪资增长率分析

In [ ]:
# 计算薪资增长率
df_sorted = df.sort_values('YearsExperience')
df_sorted['Salary_Growth_Rate'] = df_sorted['Salary'].pct_change() * 100

# 移除第一个NaN值
growth_rate = df_sorted['Salary_Growth_Rate'].dropna()

print(f"平均薪资增长率: {growth_rate.mean():.2f}%")
print(f"最高薪资增长率: {growth_rate.max():.2f}%")
print(f"最低薪资增长率: {growth_rate.min():.2f}%")

In [ ]:
# 薪资增长率随经验变化
plt.figure(figsize=(12, 6))
plt.plot(df_sorted['YearsExperience'].iloc[1:], growth_rate, marker='o', linewidth=2)
plt.title('薪资增长率随工作经验变化')
plt.xlabel('工作经验（年）')
plt.ylabel('薪资增长率 (%)')
plt.axhline(y=growth_rate.mean(), color='red', linestyle='--', label=f'平均值: {growth_rate.mean():.2f}%')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 5. 预测模型构建

In [ ]:
# 准备数据
X = df[['YearsExperience']].values
y = df['Salary'].values

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 训练线性回归模型
lr = LinearRegression()
lr.fit(X_train, y_train)

# 训练随机森林模型
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 预测
y_pred_lr = lr.predict(X_test)
y_pred_rf = rf.predict(X_test)

# 模型性能评估
print("线性回归模型性能:")
print(f"均方误差 (MSE): {mean_squared_error(y_test, y_pred_lr):.2f}")
print(f"均方根误差 (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_lr)):.2f}")
print(f"平均绝对误差 (MAE): {mean_absolute_error(y_test, y_pred_lr):.2f}")
print(f"R²分数: {r2_score(y_test, y_pred_lr):.4f}")

print("\n随机森林模型性能:")
print(f"均方误差 (MSE): {mean_squared_error(y_test, y_pred_rf):.2f}")
print(f"均方根误差 (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_rf)):.2f}")
print(f"平均绝对误差 (MAE): {mean_absolute_error(y_test, y_pred_rf):.2f}")
print(f"R²分数: {r2_score(y_test, y_pred_rf):.4f}")

In [ ]:
# 线性回归方程
slope = lr.coef_[0]
intercept = lr.intercept_
print(f"线性回归方程: 薪资 = ${slope:,.2f} × 经验 + ${intercept:,.2f}")
print(f"每增加1年经验，薪资平均增长: ${slope:,.2f}")

In [ ]:
# 预测结果可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 线性回归
axes[0].scatter(X_test, y_test, alpha=0.6, label='实际值')
axes[0].plot(X_test, y_pred_lr, color='red', linewidth=2, label='预测值')
axes[0].set_title('线性回归预测结果')
axes[0].set_xlabel('工作经验（年）')
axes[0].set_ylabel('薪资')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 随机森林
axes[1].scatter(X_test, y_test, alpha=0.6, label='实际值')
axes[1].scatter(X_test, y_pred_rf, color='red', alpha=0.6, label='预测值')
axes[1].set_title('随机森林预测结果')
axes[1].set_xlabel('工作经验（年）')
axes[1].set_ylabel('薪资')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 残差分析
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 线性回归残差
residuals_lr = y_test - y_pred_lr
axes[0].scatter(y_pred_lr, residuals_lr, alpha=0.6)
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_title('线性回归残差图')
axes[0].set_xlabel('预测值')
axes[0].set_ylabel('残差')
axes[0].grid(True, alpha=0.3)

# 随机森林残差
residuals_rf = y_test - y_pred_rf
axes[1].scatter(y_pred_rf, residuals_rf, alpha=0.6)
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_title('随机森林残差图')
axes[1].set_xlabel('预测值')
axes[1].set_ylabel('残差')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6. 薪资预测示例

In [ ]:
# 预测不同经验的薪资
experience_to_predict = np.array([[1, 3, 5, 7, 10]]).T
salary_predictions = lr.predict(experience_to_predict)

print("薪资预测示例:")
for exp, salary in zip(experience_to_predict.flatten(), salary_predictions):
    print(f"{exp}年经验: ${salary:,.2f}")

## 数据场景

### 1. 薪资制定
- **应用场景**: 根据员工经验制定合理的薪资标准
- **价值**: 确保薪酬公平性，提高员工满意度和留存率

### 2. 招聘预算规划
- **应用场景**: 根据所需经验预测招聘成本
- **价值**: 合理规划人力资源预算

### 3. 职业发展指导
- **应用场景**: 向员工展示薪资增长预期
- **价值**: 激励员工提升技能和经验

### 4. 市场薪酬分析
- **应用场景**: 对比市场薪资水平，评估竞争力
- **价值**: 制定有竞争力的薪酬策略

### 5. 成本控制
- **应用场景**: 预测不同经验组合的薪酬成本
- **价值**: 优化人员配置，控制人力成本

### 6. 薪酬谈判
- **应用场景**: 为薪酬谈判提供数据支持
- **价值**: 基于数据的透明沟通，减少争议

## 主要发现

1. **强相关性**: 工作经验与薪资呈现高度正相关
2. **线性关系**: 薪资随经验呈近似线性增长
3. **增长趋势**: 早期经验增长对薪资影响更显著
4. **模型表现**: 线性回归模型能够很好地解释薪资变化
5. **预测精度**: 模型具有较好的预测能力

## 数据质量
- 数据完整性: 无缺失值
- 数据准确性: 数据质量良好
- 数据时效性: 需要定期更新以反映当前市场状况

## 建议

1. **薪酬体系**: 基于经验建立分级薪酬体系
2. **招聘策略**: 重点吸引有经验的候选人
3. **员工发展**: 提供培训和晋升机会
4. **市场跟踪**: 定期更新市场薪资数据
5. **模型优化**: 收集更多特征（如学历、技能等）提高预测精度
6. **弹性调整**: 考虑其他因素，实现灵活的薪酬制度